In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from pandas import DataFrame
from pandas_gbq import read_gbq
from parakeet.backend.pandas.stats.stratification import stratification
from parakeet.backend.pandas.stats.frequency import frequency
from parakeet.backend.pandas.dataset import PandasDataset

## Dummy

In [5]:
df = DataFrame(
    {
        "Model": ["Chevy", "Chevy", "Ford", "Ford", "Ford", "Ford"],
        "Year": [1990, 1990, 1990, 1990, 1991, 1991],
        "Sale": [1, 0, 1, 1, 1, 0],
    }
)

In [6]:
%%time
r = stratification(df, ["Model"], "Sale")

CPU times: user 10.8 ms, sys: 1.61 ms, total: 12.4 ms
Wall time: 18.4 ms


In [7]:
r.display()

,Count,Zeros,Ones,Zeros Pct,Ones Pct,Ones Ratio,Woe,Iv
Model,,,,,,,,
Chevy,2,1,1,50.0%,25.0%,50.0%,0.000,0.000
Ford,4,1,3,50.0%,75.0%,75.0%,-1.099,0.275
Total,6,2,4,100.0%,100.0%,66.7%,-0.693,-0.000


In [8]:
%%time
r2 = stratification(df, ["Model", "Year"], "Sale")

CPU times: user 2.74 ms, sys: 5.14 ms, total: 7.88 ms
Wall time: 8.82 ms


/home/apsv/.cache/pypoetry/virtualenvs/parakeet-NSHte_Jn-py3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
r2.display()

## Frequency

In [25]:
pds = PandasDataset(df)
f = frequency(pds, ["Model"])

In [27]:
from parakeet.core.order import OrderBy
f.display(order_by=OrderBy.from_str('Frequency DESC'))

,Model,Frequency,Percentage,Cumulative Frequency,Cumulative Percentage
1,Ford,4,66.67%,4,66.67%
0,Chevy,2,33.33%,6,100.00%


## BQ Example

In [8]:
table_id = 'open-politics-344801.open_politics.base_asset'

In [9]:
%%time
data = read_gbq(f"""
SELECT loc_state,
       election_year,
       asset_type,
       IF(asset_value >= 100000, 1, 0) AS high_value,
  FROM `{table_id}`
""", project_id="open-politics-344801")

/home/apsv/.cache/pypoetry/virtualenvs/parakeet-NSHte_Jn-py3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2155: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


CPU times: user 14.1 s, sys: 1.47 s, total: 15.6 s
Wall time: 1min 55s


In [10]:
data.head()

,loc_state,election_year,asset_type,high_value
0,AL,2014,Prédio residencial,1
1,ES,2014,Prédio residencial,1
2,MS,2014,Prédio residencial,1
3,MG,2014,Prédio residencial,0
4,MG,2014,Prédio residencial,0


In [11]:
%%time
bq1 = stratification(data, ["loc_state"], "high_value")

CPU times: user 305 ms, sys: 38.9 ms, total: 344 ms
Wall time: 343 ms


In [12]:
bq1.display()

,Count,Zeros,Ones,Zeros Pct,Ones Pct,Ones Ratio,Woe,Iv
loc_state,,,,,,,,
AC,"19,497","15,166","4,331",0.4%,0.6%,22.2%,1.253,-0.002
AL,"38,606","31,506","7,100",0.9%,1.0%,18.4%,1.490,-0.001
AM,"51,393","41,528","9,865",1.2%,1.3%,19.2%,1.437,-0.002
AP,"12,600","9,769","2,831",0.3%,0.4%,22.5%,1.239,-0.001
BA,"243,027","202,363","40,664",5.9%,5.5%,16.7%,1.605,0.006
CE,"113,009","94,997","18,012",2.8%,2.5%,15.9%,1.663,0.006
DF,"20,235","13,985","6,250",0.4%,0.9%,30.9%,0.805,-0.004
ES,"78,873","63,250","15,623",1.9%,2.1%,19.8%,1.398,-0.004
GO,"186,957","146,178","40,779",4.3%,5.6%,21.8%,1.277,-0.016


### 3 cols, 

In [13]:
%%time
bq2 = stratification(data, ["election_year", "loc_state"], "high_value")

CPU times: user 398 ms, sys: 0 ns, total: 398 ms
Wall time: 405 ms


In [14]:
bq2.display()